<h1><b>Importing Libraries</b></h1>

In [ ]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         os.path.join(dirname, filename)

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from collections import Counter
from tqdm import tqdm
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import recall_score, precision_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
#from imblearn.metrics import geometric_mean_score
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import torch
from torchvision import datasets

<h1><b>Datset Loading</b></h1>

In [ ]:
dataset_path = '/kaggle/input/50-skin-disease/Best_50_class'

In [ ]:
dataset = datasets.ImageFolder(root= dataset_path)
class_names = dataset.classes
print(class_names)

In [ ]:
# dataset.targets has numeric labels for each image
counts = Counter(dataset.targets)

# Map counts to class names
for class_idx, count in counts.items():
    print(f"{dataset.classes[class_idx]}: {count} images")

In [ ]:
df_counts = pd.DataFrame({
    "Class": [dataset.classes[idx] for idx in counts.keys()],
    "Count": [counts[idx] for idx in counts.keys()]
})

# Save to CSV
df_counts.to_csv("class_counts.csv", index=False)

print("Counts saved to class_counts.csv")

<h1><b>Vision Transformer (ViT)-Tensforflow</b></h1>

<h2><b>Dataset Splitting</b></h2>

In [ ]:
import tensorflow as tf

data_dir = dataset_path
img_size = (224, 224)
batch_size = 32

# =========================
# Step 1: 70% training set
# =========================
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.30,   # leave 30% for val+test
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# =========================
# Step 2: 30% val+test pool
# =========================
val_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.30,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

# =========================
# Step 3: Split val_test_ds into 15% val + 15% test
# =========================
val_test_size = val_test_ds.cardinality().numpy()
val_size = val_test_size // 2
test_size = val_test_size - val_size  # handles odd numbers safely

val_ds = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
batch_size = 32  # Use the same batch size you set

def dataset_size(dataset):
    # Get number of batches
    batches = dataset.cardinality().numpy()
    if batches == tf.data.INFINITE_CARDINALITY or batches == tf.data.UNKNOWN_CARDINALITY:
        return "Unknown size"
    else:
        return batches * batch_size

print("Train set size:", dataset_size(train_ds))
print("Validation set size:", dataset_size(val_ds))
print("Test set size:", dataset_size(test_ds))

<h2><b>Model Compilation</b></h2>

In [ ]:
!pip install vit-keras

In [ ]:
from vit_keras import vit
from tensorflow.keras import layers, models, optimizers

# =========================
# Configuration
# =========================
num_classes = len(class_names)
image_size = 224  # ViT was pretrained on 224x224
learning_rate = 1e-4

# =========================
# Backbone (ViT)
# =========================
# vit-keras does not allow include_top=False with pretrained=True
# Instead, load with pretrained weights, then remove the head manually.
vit_base = vit.vit_b16(
    image_size=image_size,
    pretrained=True,
    include_top=True,     # must be True if pretrained=True
    pretrained_top=True   # load the pretrained head
)

# Remove the classification head (last layer)
vit_base = models.Model(
    inputs=vit_base.input,
    outputs=vit_base.layers[-2].output   # take features before final Dense
)

# Freeze the backbone
vit_base.trainable = False

# =========================
# Custom classification head
# =========================
x = vit_base.output
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=vit_base.input, outputs=output)

# =========================
# Compile
# =========================
model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
model.summary()

In [ ]:
#sections to modify
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor
    patience=5,              # Number of epochs with no improvement to wait
    restore_best_weights=True  # Restore model weights from the epoch with best val_loss
)

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    'best_model_vit.h5',        # File path to save the model
    monitor='val_loss',     # Metric to monitor
    save_best_only=True,    # Save only when improvement
    verbose=1
)

<h2><b>Model Training</b></h2>

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping, checkpoint]   # Include both callbacks here
)


<h2><b>Results</b></h2>

In [ ]:
hist_training_vit=pd.DataFrame(history.history)
hist_training_vit
# Save to CSV
hist_training_vit.to_csv("hist_training_vit.csv", index=False)
print("ViT Training history to hist_training_vit.csv")

In [ ]:
#sections to modify
fig, axs = plt.subplots(1, 2, figsize=(20, 6))

num_epochs = len(history.history['accuracy'])  # total epochs trained

# X ticks positions (integers from 0 to num_epochs, step 4)
xticks = range(0, num_epochs + 1, 4)

# Plot accuracy
axs[0].plot(history.history['accuracy'], label='Train Accuracy')
axs[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axs[0].set_xlabel('Epoch', fontsize=20)
axs[0].set_ylabel('Accuracy', fontsize=20)
axs[0].set_title('Training and Validation Accuracy - ViT', fontsize=22)
axs[0].legend(fontsize=18)
axs[0].set_xticks(xticks)
axs[0].tick_params(axis='x', labelsize=18)
axs[0].tick_params(axis='y', labelsize=18)
axs[0].grid(True)

# Plot loss
axs[1].plot(history.history['loss'], label='Train Loss')
axs[1].plot(history.history['val_loss'], label='Validation Loss')
axs[1].set_xlabel('Epoch', fontsize=20)
axs[1].set_ylabel('Loss', fontsize=20)
axs[1].set_title('Training and Validation Loss - ViT', fontsize=22)
axs[1].legend(fontsize=18)
axs[1].set_xticks(xticks)
axs[1].tick_params(axis='x', labelsize=18)
axs[1].tick_params(axis='y', labelsize=18)
axs[1].grid(True)

plt.tight_layout()

# Save the combined figure
plt.savefig('vit_training_curves.png', dpi=600)
plt.savefig('vit_training_curves.pdf')

plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef

# 1. Get true and predicted labels
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 2. Compute confusion matrix and metrics
cm = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
mcc = matthews_corrcoef(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f'Matthews Correlation Coefficient: {mcc:.4f}')

# 3. Plot confusion matrix
plt.figure(figsize=(30, 25))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels= class_names, yticklabels= class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig('vit_confusion_matrix.png', dpi=600)
plt.savefig('vit_confusion_matrix.pdf')
plt.show()

# 4. Save metrics to CSV
metrics = {
    'Accuracy': [accuracy],
    'Recall': [recall],
    'Precision': [precision],
    'F1 Score': [f1],
    'MCC':[mcc]
}
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
df_cm.to_csv("confusion_vit.csv")

df_metrics_vit = pd.DataFrame(metrics)
df_metrics_vit.to_csv('performance_metrics_vit.csv', index=False)

<h1><b>Swin Transformer</b></h1>

<h2><b>Importing Librarires</b></h2>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef
import copy
import os

<h2><b>Dataset Loading</b></h2>

In [ ]:
data_dir = "/kaggle/input/50-skin-disease/Best_50_class"   # dataset root folder (class subfolders inside)
img_size = 224            # Swin expects 224x224
batch_size = 32
epochs = 50
learning_rate = 1e-4
seed = 123
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms   # ✅ this line

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # ImageNet normalization
        std=[0.229, 0.224, 0.225]
    )
])

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
num_classes = len(full_dataset.classes)

<h2><b>Dataset Splitting</b></h2>

In [ ]:
dataset_size = len(full_dataset)
train_size = int(0.7 * dataset_size)
val_size   = int(0.15 * dataset_size)
test_size  = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(seed)
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")

In [ ]:
# full dataset
full_dataset = datasets.ImageFolder(data_dir, transform=transform)

# save class names once
class_names = full_dataset.classes   # ✅ store here

<h2>Model Compiling</h2>

In [ ]:
from torchvision import models

model = models.swin_t(weights="IMAGENET1K_V1")  # Swin Tiny pretrained
model.head = nn.Linear(model.head.in_features, num_classes)  # replace classifier
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

<h2><b>Model Training</b></h2>

In [ ]:
# =========================
# Early Stopping + Checkpoint
# =========================
best_val_loss = float("inf")
best_model_wts = copy.deepcopy(model.state_dict())
patience = 10
counter = 0
checkpoint_path = "best_swin_model.pth"

num_epochs = 10
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # ---- Validation ----
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = running_loss / len(val_loader.dataset)
    val_acc = 100. * correct / total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% "
          f"| Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # ---- Checkpoint + Early Stopping ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), checkpoint_path)
        print(f"✅ Best model updated & saved at epoch {epoch+1}")
        counter = 0
    else:
        counter += 1
        print(f"⚠️ No improvement for {counter} epochs")

        if counter >= patience:
            print("⏹️ Early stopping triggered")
            break

# Load best model weights
model.load_state_dict(best_model_wts)
print("✅ Loaded best model from training")

# =========================
# Save Training History
# =========================
history = {
    'loss': train_losses,
    'val_loss': val_losses,
    'accuracy': train_accuracies,
    'val_accuracy': val_accuracies
}
df_history = pd.DataFrame(history)
df_history.to_csv("swin_training_history.csv", index=False)
print("📁 Training history saved to swin_training_history.csv")

<h2><b>Results</b></h2>

In [ ]:
# =========================
# Plot Training Curves
# =========================
fig, axs = plt.subplots(1, 2, figsize=(20, 6))
num_epochs_done = len(train_losses)
xticks = range(0, num_epochs_done + 1, 2)

# Accuracy
axs[0].plot(train_accuracies, label='Train Accuracy')
axs[0].plot(val_accuracies, label='Validation Accuracy')
axs[0].set_xlabel('Epoch', fontsize=20)
axs[0].set_ylabel('Accuracy (%)', fontsize=20)
axs[0].set_title('Training and Validation Accuracy - Swin', fontsize=22)
axs[0].legend(fontsize=18)
axs[0].set_xticks(xticks)
axs[0].grid(True)

# Loss
axs[1].plot(train_losses, label='Train Loss')
axs[1].plot(val_losses, label='Validation Loss')
axs[1].set_xlabel('Epoch', fontsize=20)
axs[1].set_ylabel('Loss', fontsize=20)
axs[1].set_title('Training and Validation Loss - Swin', fontsize=22)
axs[1].legend(fontsize=18)
axs[1].set_xticks(xticks)
axs[1].grid(True)

plt.tight_layout()
plt.savefig('swin_training_curves.png', dpi=600)
plt.savefig('swin_training_curves.pdf')
plt.show()

In [ ]:
# =========================
# Test Evaluation
# =========================
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Performance Metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
mcc = matthews_corrcoef(y_true, y_pred)

print("\n===== 📊 Test Performance =====")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"MCC:       {mcc:.4f}")

# Save Performance Metrics
metrics_df = pd.DataFrame([{
    "Accuracy": acc,
    "Precision": prec,
    "Recall": rec,
    "F1-Score": f1,
    "MCC": mcc
}])
metrics_df.to_csv("swin_test_metrics.csv", index=False)
print("📁 Test metrics saved to swin_test_metrics.csv")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Evaluate on test set
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(30, 25))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig('swin_confusion_matrix.png', dpi=600)
plt.savefig('swin_confusion_matrix.pdf')
plt.show()

df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
df_cm.to_csv("confusion_swin.csv")

# Classification report
print(classification_report(y_true, y_pred, target_names=class_names))


<h2><b>Plotting Predictions</b></h2>

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models

# =========================
# 1) Rebuild model + load checkpoint
# =========================
num_classes = len(class_names)
model = models.swin_t(weights="IMAGENET1K_V1")
model.head = nn.Linear(model.head.in_features, num_classes)
model = model.to(device)

checkpoint_path = "/kaggle/input/swin_10epochs/pytorch/default/1/best_swin_model.pth"   # or your .h file
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()
print("✅ Model loaded from checkpoint")

# =========================
# 2) Run inference on test set
# =========================
correct_samples = []
wrong_samples = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        for img, true, pred in zip(images, labels, preds):
            if true == pred and len(correct_samples) < 4:
                correct_samples.append((img.cpu(), true.item(), pred.item()))
            elif true != pred and len(wrong_samples) < 4:
                wrong_samples.append((img.cpu(), true.item(), pred.item()))

        # stop once we got enough
        if len(correct_samples) >= 4 and len(wrong_samples) >= 4:
            break

# =========================
# 3) Helper to unnormalize image
# =========================
def imshow(img_tensor, title):
    img = img_tensor.numpy().transpose((1, 2, 0))
    img = np.clip(img, 0, 1)   # assuming transforms.ToTensor() only
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")

# =========================
# 4) Plot results
# =========================
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, (img, true, pred) in enumerate(correct_samples):
    plt.subplot(2, 4, i+1)
    imshow(img, f"✅ True: {class_names[true]}\nPred: {class_names[pred]}")

for i, (img, true, pred) in enumerate(wrong_samples):
    plt.subplot(2, 4, i+5)
    imshow(img, f"❌ True: {class_names[true]}\nPred: {class_names[pred]}")

plt.tight_layout()
plt.show()


In [ ]:
import random
import matplotlib.pyplot as plt

# =========================
# Collect Predictions
# =========================
model.eval()
all_images, all_labels, all_preds = [], [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Collecting predictions"):
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)
        
        all_images.extend(images.cpu())   # keep CPU tensors for plotting
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# =========================
# Visualization Function
# =========================
def plot_class_predictions(class_idx, correct=True, num_samples=4, save_path=None):
    """Plot correct or incorrect predictions for a given class index"""
    imgs, titles = [], []

    for img, label, pred in zip(all_images, all_labels, all_preds):
        if label == class_idx:
            if correct and label == pred:   # correct prediction
                imgs.append(img)
                titles.append((label, pred))
            elif not correct and label != pred:  # incorrect prediction
                imgs.append(img)
                titles.append((label, pred))
        if len(imgs) >= num_samples:
            break
    
    if len(imgs) == 0:
        print(f"No {'correct' if correct else 'incorrect'} predictions found for class {class_names[class_idx]}")
        return

    # Plot
    fig, axes = plt.subplots(1, len(imgs), figsize=(15, 4))
    fig.suptitle(
        f"{'Correct' if correct else 'Wrong'} Predictions: {class_names[class_idx]}",
        fontsize=16, weight="bold"
    )

    for ax, img, (label, pred) in zip(axes, imgs, titles):
        img = img.permute(1, 2, 0).numpy()
        img = (img - img.min()) / (img.max() - img.min())  # normalize for display
        ax.imshow(img)
        ax.axis("off")
        ax.set_title(f"Actual: {class_names[label]}\nPred: {class_names[pred]}",
                     fontsize=10)

    if save_path:
        fig.savefig(save_path, dpi=600, bbox_inches="tight")  # save before plt.show()

    plt.show()



# =========================
# Example Usage
# =========================
# Suppose you want to check class index 5
selected_class = 5

# Save correct predictions
plot_class_predictions(selected_class, correct=True, save_path="swin_correct.pdf")

# Save wrong predictions
plot_class_predictions(selected_class, correct=False, save_path="swin_wrong.pdf")

<h2><b>Grad Cam Analysis</b></h2>

In [ ]:
pip install grad-cam

In [ ]:
def visualize_gradcam(model, img_path, transform, device, class_names, target_layer, save_path=None, format='png'):
    # Load and preprocess image
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    # Initialize Grad-CAM
    gradcam = GradCAM(model, target_layer)
    
    # Generate heatmap
    heatmap, pred_class = gradcam.generate(img_tensor)
    
    # Convert tensor to image
    img_resized = img.resize((224, 224))
    img_array = np.array(img_resized) / 255.0
    
    # Apply heatmap to image
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    heatmap = heatmap[..., ::-1]  # Convert BGR to RGB
    
    # Superimpose heatmap on original image
    cam_img = heatmap + np.float32(img_array)
    cam_img = cam_img / np.max(cam_img)
    
    # Plot results
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(img_array)
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(heatmap)
    axes[1].set_title('Heatmap')
    axes[1].axis('off')
    
    axes[2].imshow(cam_img)
    axes[2].set_title(f'Grad-CAM: {class_names[pred_class]}')
    axes[2].axis('off')
    
    plt.tight_layout()
    
    # Save the figure if save_path is provided
    if save_path:
        if format.lower() == 'pdf':
            plt.savefig(save_path, format='pdf', bbox_inches='tight', dpi=300)
        else:
            plt.savefig(save_path, format='png', bbox_inches='tight', dpi=300)
        print(f"Saved visualization to {save_path}")
    
    plt.show()
    plt.close(fig)  # Close the figure to free memory
    
    return pred_class

# Now you can call it with save_path
test_image_path = "/kaggle/input/50-skin-disease/Best_50_class/Measles/Measles MSLD V2/mpx_fold1_Test_MSL_16_01.jpg"
pred_class = visualize_gradcam(
    model, 
    test_image_path, 
    transform, 
    device, 
    class_names, 
    target_layer,
    save_path="/kaggle/working/gradcam_singleImagevisualization.png",  # This should work now
    format='png'
)

In [ ]:
def visualize_gradcam_grid(model, dataloader, device, class_names, target_layer, num_images=9, save_path=None, format='png'):
    model.eval()
    
    # Get a batch of images
    images, labels = next(iter(dataloader))
    images = images[:num_images].to(device)
    labels = labels[:num_images].cpu().numpy()
    
    # Initialize Grad-CAM
    gradcam = GradCAM(model, target_layer)
    
    # Create figure
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.ravel()
    
    for i in range(num_images):
        # Generate heatmap
        heatmap, pred_class = gradcam.generate(images[i:i+1])
        
        # Convert tensor to image
        img = images[i].cpu().permute(1, 2, 0).numpy()
        img = (img - img.min()) / (img.max() - img.min())
        
        # Apply heatmap to image
        heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255
        heatmap = heatmap[..., ::-1]  # Convert BGR to RGB
        
        # Superimpose heatmap on original image
        cam_img = heatmap + np.float32(img)
        cam_img = cam_img / np.max(cam_img)
        
        # Plot
        axes[i].imshow(cam_img)
        axes[i].set_title(f'True: {class_names[labels[i]]}\nPred: {class_names[pred_class]}', fontsize=10)
        axes[i].axis('off')
    
    plt.tight_layout()
    
    # Save the figure if save_path is provided
    if save_path:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        if format.lower() == 'pdf':
            plt.savefig(save_path, format='pdf', bbox_inches='tight', dpi=300)
        else:
            plt.savefig(save_path, format='png', bbox_inches='tight', dpi=300)
        print(f"✅ Saved grid visualization to {save_path}")
    
    plt.show()
    plt.close(fig)  # Close the figure to free memory

# Usage example
visualize_gradcam_grid(
    model, 
    test_loader, 
    device, 
    class_names, 
    target_layer, 
    num_images=9,
    save_path="/kaggle/working/gradcam_resultsGridImage.png",
    format='png'
)

Another Approach:

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

# --------------------
# Load your model
# --------------------
num_classes = len(class_names)
model = models.swin_t(weights="IMAGENET1K_V1")
model.head = nn.Linear(model.head.in_features, num_classes)
model = model.to(device)

checkpoint_path = "/kaggle/input/swin_10epochs/pytorch/default/1/best_swin_model.pth"   # or your .h file
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()
print("✅ Model loaded from checkpoint")

In [ ]:
# Print model structure to find the right layer
print(model.features)

# Try different layers - common options for Swin Transformer
# Option 1: Last layer of the feature extractor
target_layer = model.features[-1][-1]

# Option 2: Last normalization layer
for name, module in model.named_modules():
    if 'norm' in name and 'head' not in name:
        print(f"Found norm layer: {name}")
        target_layer = module

# Option 3: Last attention block
for name, module in model.named_modules():
    if 'attention' in name:
        print(f"Found attention layer: {name}")
        target_layer = module

# Register hooks
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

In [ ]:
# --------------------
# Define preprocessing
# --------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --------------------
# Grad-CAM hook setup - CORRECTED
# --------------------
activations = {}
gradients = {}

def forward_hook(module, input, output):
    activations['value'] = output

def backward_hook(module, grad_in, grad_out):
    gradients['value'] = grad_out[0]


# --------------------
# Grad-CAM function
# --------------------
def compute_gradcam(img_tensor, class_idx=None):
    img_tensor = img_tensor.to(device)

    # Forward
    output = model(img_tensor.unsqueeze(0))  # Add batch dimension
    if class_idx is None:
        class_idx = output.argmax(dim=1).item()

    # Backward
    model.zero_grad()
    output[0, class_idx].backward()

    # Get activations & gradients
    act = activations['value'].detach().cpu()    # [B, C, H, W]
    grad = gradients['value'].detach().cpu()     # [B, C, H, W]
    
    # Global average pooling of gradients
    weights = grad.mean(dim=(2, 3), keepdim=True)  # [B, C, 1, 1]
    
    # Weight the activation maps
    cam = (weights * act).sum(dim=1)  # [B, H, W]
    cam = cam[0].numpy()  # Get the first image in batch
    
    # Normalize CAM
    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-8)  # Avoid division by zero
    
    # Resize to input size
    cam = cv2.resize(cam, (224, 224))
    
    return cam, class_idx

def overlay_gradcam(cam, img_tensor):
    img = img_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    img = (img - img.min()) / (img.max() - img.min())
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    heatmap = heatmap[..., ::-1]  # Convert BGR to RGB
    
    # Superimpose heatmap on original image
    overlay = 0.5 * heatmap + 0.5 * img
    overlay = np.clip(overlay, 0, 1)
    return overlay

# --------------------
# Example usage
# --------------------
img_path = "/kaggle/input/50-skin-disease/Best_50_class/Measles/Measles MSLD V2/mpx_fold1_Test_MSL_16_01.jpg"
img = Image.open(img_path).convert("RGB")
img_tensor = transform(img)

cam, pred = compute_gradcam(img_tensor)
overlay = overlay_gradcam(cam, img_tensor)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(cam, cmap='jet')
plt.title("Grad-CAM Heatmap")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(overlay)
plt.title(f"Overlay (Predicted: {class_names[pred]})")
plt.axis('off')

plt.tight_layout()
plt.savefig("gradcam_result.png", dpi=300, bbox_inches='tight')
plt.show()

<h1><b>Vit-Pytorch</b></h2>

<h2><b>Importing Libraries</b></h2>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

<h2><b>Model Compiling</b></h2>

In [ ]:
# =========================
# Define ViT Model
# =========================
model = models.vit_b_16(weights="IMAGENET1K_V1")  # Pretrained ViT-Base, patch 16
model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)  # Replace classification head
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

<h2><b>Model Training</b></h2>

In [ ]:
# =========================
# Early Stopping + Checkpoint (same as before)
# =========================
best_val_loss = float("inf")
best_model_wts = copy.deepcopy(model.state_dict())
patience = 5
counter = 0
checkpoint_path = "best_vit_model.pth"

num_epochs = 50
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # ---- Validation ----
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = running_loss / len(val_loader.dataset)
    val_acc = 100. * correct / total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% "
          f"| Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # ---- Checkpoint + Early Stopping ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), checkpoint_path)
        print(f"✅ Best model updated & saved at epoch {epoch+1}")
        counter = 0
    else:
        counter += 1
        print(f"⚠️ No improvement for {counter} epochs")
        if counter >= patience:
            print("⏹️ Early stopping triggered")
            break

# Load best model weights
model.load_state_dict(best_model_wts)
print("✅ Loaded best ViT model from training")
# =========================
# Save Training History
# =========================
history = {
    'loss': train_losses,
    'val_loss': val_losses,
    'accuracy': train_accuracies,
    'val_accuracy': val_accuracies
}
df_history = pd.DataFrame(history)
df_history.to_csv("vit_training_history.csv", index=False)
print("📁 Training history saved to vit_training_history.csv")


<h2><b>Results</b></h2>

In [ ]:
# =========================
# Plot Training Curves
# =========================
fig, axs = plt.subplots(1, 2, figsize=(20, 6))
num_epochs_done = len(train_losses)
xticks = range(0, num_epochs_done + 1, 2)

# Accuracy
axs[0].plot(train_accuracies, label='Train Accuracy')
axs[0].plot(val_accuracies, label='Validation Accuracy')
axs[0].set_xlabel('Epoch', fontsize=20)
axs[0].set_ylabel('Accuracy (%)', fontsize=20)
axs[0].set_title('Training and Validation Accuracy - Vision Transformer', fontsize=22)
axs[0].legend(fontsize=18)
axs[0].set_xticks(xticks)
axs[0].grid(True)

# Loss
axs[1].plot(train_losses, label='Train Loss')
axs[1].plot(val_losses, label='Validation Loss')
axs[1].set_xlabel('Epoch', fontsize=20)
axs[1].set_ylabel('Loss', fontsize=20)
axs[1].set_title('Training and Validation Loss - Vision Transformer', fontsize=22)
axs[1].legend(fontsize=18)
axs[1].set_xticks(xticks)
axs[1].grid(True)

plt.tight_layout()
plt.savefig('vitTorch_training_curves.png', dpi=600)
plt.savefig('vitTorch_training_curves.pdf')
plt.show()

In [ ]:
# =========================
# Test Evaluation
# =========================
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Performance Metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
mcc = matthews_corrcoef(y_true, y_pred)

print("\n===== 📊 Test Performance =====")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"MCC:       {mcc:.4f}")

# Save Performance Metrics
metrics_df = pd.DataFrame([{
    "Accuracy": acc,
    "Precision": prec,
    "Recall": rec,
    "F1-Score": f1,
    "MCC": mcc
}])
metrics_df.to_csv("vitTorch_test_metrics.csv", index=False)
print("📁 Test metrics saved to vitTorch_test_metrics.csv")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Evaluate on test set
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(30, 25))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig('vitTorch_confusion_matrix.png', dpi=600)
plt.savefig('vitTorch_confusion_matrix.pdf')
plt.show()

df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
df_cm.to_csv("confusion_vitTorch.csv")

# Classification report
print(classification_report(y_true, y_pred, target_names=class_names))


<h1><b>Zipping all files</b></h1>

In [ ]:
!zip -r /kaggle/working/output_files_swin_vit.zip /kaggle/working/*